In [ ]:
import reset
from pathlib import Path
import numpy as np
import scipy as sp
import tifffile
import math
import plotly.graph_objects as go

In [2]:
# Assign variables from reset
animal_loc = reset.animal_loc
animal_id = reset.animal_id
hemi = reset.hemi
divisor = reset.divisor
down_x = reset.down_x
down_y = reset.down_y
loc_max = reset.loc_max
grid = reset.grid
frame_len = reset.frame_len

In [3]:
# Create list of locations as strings
loc_list = [str(n+1) for n in range(loc_max)]

# Define paths for input and output images
image_loc = animal_loc / "ffc" / hemi
save_loc = animal_loc / "xmerged" / hemi

# Ensure save location exists
save_loc.mkdir(parents=True, exist_ok=True)

image_path_list = [f"{image_loc}/{animal_id}_{hemi}_{loc}.tif" for loc in loc_list]

In [4]:
rect_x = 35
rect_x_move = 100
rect_x_start = down_x - rect_x - rect_x_move

width = down_x
frame_height = down_y
frame_width = 3 * width
frame = np.zeros((frame_len, frame_height, frame_width))

In [5]:
def find_brightest_frame(path_num1, path_num2, path_num3):
    global max_frame
    """
    Find the brightest frame across three image sequences.
    Returns the index of the brightest frame and the average brightness for each image.
    """
    average_brightness = []
    for path_num in [path_num1, path_num2, path_num3]:
        brightness_list = []
        for fn in range(frame_len):
            im = tifffile.imread(image_path_list[path_num - 1])
            im_f = im[fn, :, :]
            brightness_list.append(np.average(im_f))
        
        max_brightness = max(brightness_list)
        max_frame_index = np.argmax(brightness_list)
        average_brightness.append((max_brightness, max_frame_index))

    # Determine the maximum brightness and corresponding frame
    max_brightness_values, max_frame_indices = zip(*average_brightness)
    max_frame = max_frame_indices[np.argmax(max_brightness_values)]
    
    print(f"Brightest frame across sections: {max_frame}")
    return max_frame

In [6]:
# Define function for aligning x positions based on maximum correlation
def x_connector(path_num1, path_num2, rect_x, rect_x_move, rect_x_start):
    global start_x
    # Load and extract frames from both images
    im1 = tifffile.imread(image_path_list[path_num1 - 1])
    im1_f = im1[max_frame, :, :]
    del im1

    im2 = tifffile.imread(image_path_list[path_num2 - 1])
    im2_f = im2[max_frame, :, :]
    del im2

    # Calculate correlations over specified range
    cor = [
        np.corrcoef(
            im1_f[:, :rect_x].ravel(),
            im2_f[:, i:i + rect_x].ravel()
        )[0, 1]
        for i in range(rect_x_start, rect_x_start + rect_x_move)
    ]

    # Determine start position with max correlation
    start_x = np.argmax(cor) + rect_x

def three_sections(f1, f2, f3, frame_name, rect_x, rect_x_move, rect_x_start):
    global target1_start_x, target2_start_x

    x_connector(f1, f2, rect_x, rect_x_move, rect_x_start)
    target1_start_x = start_x
    x_connector(f2, f3, rect_x, rect_x_move, rect_x_start)
    target2_start_x = start_x

    print("First section target frame and start x:", target1_start_x)
    print("Second section target frame and start x:", target2_start_x)

def left(f1, frame_name=frame):
    for fn in range(frame_len):
        im1 = tifffile.imread(image_path_list[f1 - 1])
        rot_im1 = im1[fn, ::-1, ::-1]
        frame_name[fn, :, :width] = rot_im1
        del im1

def middle(f2, startx1, frame_name=frame):
    for fn in range(frame_len):
        im2 = tifffile.imread(image_path_list[f2 - 1])
        rot_im2 = im2[fn, ::-1, ::-1]
        
        # Handle the non-overlapping area
        frame_name[fn, :, width:2 * width - startx1] = rot_im2[:, startx1:]
        
        # Average the overlapping area between left and middle
        overlap = frame_name[fn, :, width - startx1:width]  # Left's end part
        frame_name[fn, :, width - startx1:width] = (overlap + rot_im2[:, :startx1]) / 2
        del im2

def right(f3, startx1, startx2, frame_name=frame):
    for fn in range(frame_len):
        im3 = tifffile.imread(image_path_list[f3 - 1])
        rot_im3 = im3[fn, ::-1, ::-1]
        
        # Handle the non-overlapping area
        frame_name[fn, :, 2 * width - startx1:3 * width - startx1 - startx2] = rot_im3[:, startx2:]
        
        # Average the overlapping area between middle and right
        overlap = frame_name[fn, :, 2 * width - startx1 - startx2:2 * width - startx1]  # Middle's end part
        frame_name[fn, :, 2 * width - startx1 - startx2:2 * width - startx1] = (overlap + rot_im3[:, :startx2]) / 2
        del im3

In [7]:
find_brightest_frame(1, 2, 3)
three_sections(1, 2, 3, frame, rect_x, rect_x_move, rect_x_start)

Brightest frame across sections: 29
First section target frame and start x: 109
Second section target frame and start x: 35


In [8]:
left(1)

In [9]:
middle(2, startx1=65)

In [10]:
right(3, startx1=65, startx2=133)

In [ ]:
# fig = go.Figure(data=go.Heatmap(z=frame[max_frame], colorscale="Viridis"))
# fig.update_layout(xaxis=dict(scaleanchor="y"), yaxis=dict(scaleanchor="x"))
# fig.show()

In [11]:
save_path = save_loc / f"{animal_id}_{hemi}_1.tif"
tifffile.imwrite(save_path, frame.astype('uint8'))
print(f"Aligned image saved to {save_path}")

Aligned image saved to /Volumes/BaffaloSSDPUTU3C1TB/rbak_data/rbak006/xmerged/l/rbak006_l_1.tif


In [12]:
find_brightest_frame(6, 5, 4)
three_sections(6, 5, 4, frame, rect_x, rect_x_move, rect_x_start)

Brightest frame across sections: 20
First section target frame and start x: 85
Second section target frame and start x: 134


In [13]:
left(6)
middle(5, startx1=target1_start_x)

In [14]:
right(4, startx1=target1_start_x, startx2=20)

In [ ]:
# fig = go.Figure(data=go.Heatmap(z=frame[max_frame], colorscale="Viridis"))
# fig.update_layout(xaxis=dict(scaleanchor="y"), yaxis=dict(scaleanchor="x"))
# fig.show()

In [15]:
save_path = save_loc / f"{animal_id}_{hemi}_2.tif"
tifffile.imwrite(save_path, frame.astype('uint8'))
print(f"Aligned image saved to {save_path}")

Aligned image saved to /Volumes/BaffaloSSDPUTU3C1TB/rbak_data/rbak006/xmerged/l/rbak006_l_2.tif


In [16]:
find_brightest_frame(7, 8, 9)
# Run alignment on specified sections
three_sections(7, 8, 9, frame, rect_x, rect_x_move, rect_x_start)

Brightest frame across sections: 45
First section target frame and start x: 122
Second section target frame and start x: 90


In [17]:
left(7)
middle(8, startx1=75)

In [18]:
right(9, startx1=75, startx2=target2_start_x)

In [ ]:
# fig = go.Figure(data=go.Heatmap(z=frame[max_frame], colorscale="Viridis"))
# fig.update_layout(xaxis=dict(scaleanchor="y"), yaxis=dict(scaleanchor="x"))
# fig.show()

In [19]:
# Save the aligned image
save_path = save_loc / f"{animal_id}_{hemi}_4.tif"
tifffile.imwrite(save_path, frame.astype('uint8'))
print(f"Aligned image saved to {save_path}")

Aligned image saved to /Volumes/BaffaloSSDPUTU3C1TB/rbak_data/rbak006/xmerged/l/rbak006_l_4.tif


In [20]:
find_brightest_frame(12, 11, 10)
three_sections(12, 11, 10, frame, rect_x, rect_x_move, rect_x_start)

Brightest frame across sections: 60
First section target frame and start x: 76
Second section target frame and start x: 134


In [21]:
left(12)
middle(11, startx1=80)

In [22]:
right(10, startx1=80, startx2=30)

In [ ]:
# fig = go.Figure(data=go.Heatmap(z=frame[max_frame], colorscale="Viridis"))
# fig.update_layout(xaxis=dict(scaleanchor="y"), yaxis=dict(scaleanchor="x"))
# fig.show()

In [23]:
save_path = save_loc / f"{animal_id}_{hemi}_5.tif"
tifffile.imwrite(save_path, frame.astype('uint8'))
print(f"Aligned image saved to {save_path}")

Aligned image saved to /Volumes/BaffaloSSDPUTU3C1TB/rbak_data/rbak006/xmerged/l/rbak006_l_5.tif


In [24]:
find_brightest_frame(13, 14, 15)
three_sections(13, 14, 15, frame, rect_x, rect_x_move, rect_x_start)

Brightest frame across sections: 30
First section target frame and start x: 103
Second section target frame and start x: 90


In [25]:
left(13)
middle(14, startx1=50)

In [26]:
right(15, startx1=50, startx2=target2_start_x)

In [ ]:
# fig = go.Figure(data=go.Heatmap(z=frame[max_frame], colorscale="Viridis"))
# fig.update_layout(xaxis=dict(scaleanchor="y"), yaxis=dict(scaleanchor="x"))
# fig.show()

In [27]:
save_path = save_loc / f"{animal_id}_{hemi}_3.tif"
tifffile.imwrite(save_path, frame.astype('uint8'))
print(f"Aligned image saved to {save_path}")

Aligned image saved to /Volumes/BaffaloSSDPUTU3C1TB/rbak_data/rbak006/xmerged/l/rbak006_l_3.tif


In [28]:
find_brightest_frame(16, 17, 18)
three_sections(16, 17, 18, frame, rect_x, rect_x_move, rect_x_start)

Brightest frame across sections: 58
First section target frame and start x: 133
Second section target frame and start x: 114


In [29]:
left(16)
middle(17, startx1=35)

In [30]:
right(18, startx1=35, startx2=50)

In [ ]:
# fig = go.Figure(data=go.Heatmap(z=frame[max_frame], colorscale="Viridis"))
# fig.update_layout(xaxis=dict(scaleanchor="y"), yaxis=dict(scaleanchor="x"))
# fig.show()

In [31]:
save_path = save_loc / f"{animal_id}_{hemi}_6.tif"
tifffile.imwrite(save_path, frame.astype('uint8'))
print(f"Aligned image saved to {save_path}")

Aligned image saved to /Volumes/BaffaloSSDPUTU3C1TB/rbak_data/rbak006/xmerged/l/rbak006_l_6.tif
